In [ ]:
# Don't use pls

# import requests

# url = "https://publicapi.traffy.in.th/dump-csv-chadchart/bangkok_traffy.csv"
# response = requests.get(url)

# if response.status_code == 200:
#     with open("bangkok_traffy.csv", "wb") as f:
#         f.write(response.content)
#     print("Data downloaded successfully.")
# else:
#     print(f"Failed to download data. Status code: {response.status_code}")

In [ ]:
# !pip install pythainlp -q
# !pip install --no-cache-dir transformers sentencepiece -q

In [ ]:
# !gdown 1uNAYBPerV6IaiWXLPF4Efkxb4Gg-U4Cz

Downloading...
From: https://drive.google.com/uc?id=1uNAYBPerV6IaiWXLPF4Efkxb4Gg-U4Cz
To: /content/bangkok_traffy.csv
100% 362M/362M [00:04<00:00, 87.8MB/s]


In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

# Load Data

In [ ]:
df = pd.read_csv("../data/clean_with_spark.csv")

In [ ]:
# df.head()

# Select and Clean feature & target columns

In [ ]:
df = df[["type", "comment", "province"]]

In [ ]:
df = df[df['province'].str.contains('กรุงเทพ|bangkok',na=False,case=False)]
df = df.drop(["province"], axis=1)

In [ ]:
df['type'].dropna(axis=0, inplace=True)
df['comment'].dropna(axis=0, inplace=True)

In [ ]:
df['type'].fillna('', inplace=True) 
df['type'] = df['type'].astype(str) 
df['type'] = df['type'].str.strip('{}').apply(lambda x :[e for e in x.split(',') if e])

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

,type,comment
0,[],ไม่มีภาพ
1,[ความสะอาด],ขยะเยอะ
2,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
...,...,...
273112,[ความสะอาด],ไม่มีเจ้าหน้าที่มาเก็บขยะหลายวันแล้วคะ
273113,"[ถนน, คลอง]",* ปัญหา: ประชาชนพบจระเข้ 2 ตัว ขอให้ดำเนินการจ...
273114,[ถนน],* ปัญหา: ประชาชนต้องการให้เจ้าหน้าที่มาเก็บขยะ...
273115,[],ผมขับมอเตอร์ไซค์ผ่าน บริเวณสายสอง มีกลุ่มควันเ...


Preprocess: type

In [ ]:
df["type"] = df["type"].apply(lambda x: np.nan if x == [] else x)
df = df.dropna().reset_index(drop=True)
df

,type,comment
0,[ความสะอาด],ขยะเยอะ
1,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
2,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
3,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
4,"[ถนน, ร้องเรียน, น้ำท่วม, ความปลอดภัย]",1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...
...,...,...
208994,"[ถนน, ท่อระบายน้ำ]",มีฝาท่อถนนเพชรบุรี ตั้งแต่ก่อนถึงทางรถไฟ ไปจนถ...
208995,[ความสะอาด],ไม่มีเจ้าหน้าที่มาเก็บขยะหลายวันแล้วคะ
208996,"[ถนน, คลอง]",* ปัญหา: ประชาชนพบจระเข้ 2 ตัว ขอให้ดำเนินการจ...
208997,[ถนน],* ปัญหา: ประชาชนต้องการให้เจ้าหน้าที่มาเก็บขยะ...


In [ ]:
def apply_1(x):
        if (len(x) == 1) and (["ร้องเรียน"] in x):
            x = ["other"]
        else:
            x = [elem for elem in x if elem not in ["ร้องเรียน"]]
        return x
def preprocess_type(df):
    df = df.copy()
    other_list = ["PM2.5", "สอบถาม", "เสนอแนะ", "คนจรจัด", "การเดินทาง", "ห้องน้ำ", "ป้ายจราจร"]
    df['type'] = df['type'].progress_apply(lambda x: [elem if elem not in other_list else 'other' for elem in x])
    df["type"] = df["type"].apply(lambda x: ["other"] if x == [] else x)
    df['type'] = df['type'].progress_apply(apply_1)
    df["type"] = df["type"].apply(lambda x: ["other"] if x == [] else x)
    return df

def apply_filter_type(x):
    order_list = ['สัตว์จรจัด',
 'other',
 'ป้าย',
 'ต้นไม้',
 'คลอง',
 'เสียงรบกวน',
 'สายไฟ',
 'จราจร',
 'สะพาน',
 'ท่อระบายน้ำ',
 'กีดขวาง',
 'น้ำท่วม',
 'ความสะอาด',
 'ความปลอดภัย',
 'แสงสว่าง',
 'ทางเท้า',
 'ถนน']
    for order in order_list:
        if order in x:
            return order
df = preprocess_type(df)
df["type"] = df["type"].apply(apply_filter_type)
df.head()

  0%|          | 0/208999 [00:00<?, ?it/s]

  0%|          | 0/208999 [00:00<?, ?it/s]

,type,comment
0,ความสะอาด,ขยะเยอะ
1,สายไฟ,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
2,สะพาน,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
3,น้ำท่วม,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
4,น้ำท่วม,1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...


In [ ]:
type_columns = ['ถนน',
 'ทางเท้า',
 'แสงสว่าง',
 'ความปลอดภัย',
 'ความสะอาด',
 'น้ำท่วม',
 'กีดขวาง',
 'ท่อระบายน้ำ',
 'สะพาน',
 'จราจร',
 'สายไฟ',
 'เสียงรบกวน',
 'คลอง',
 'ต้นไม้',
 'ป้าย',
 'other',
 'สัตว์จรจัด']

type_to_idx = {col: index for index, col in enumerate(type_columns)}
idx_to_type = {index: col for index, col in enumerate(type_columns)}
type_to_idx

{'ถนน': 0,
 'ทางเท้า': 1,
 'แสงสว่าง': 2,
 'ความปลอดภัย': 3,
 'ความสะอาด': 4,
 'น้ำท่วม': 5,
 'กีดขวาง': 6,
 'ท่อระบายน้ำ': 7,
 'สะพาน': 8,
 'จราจร': 9,
 'สายไฟ': 10,
 'เสียงรบกวน': 11,
 'คลอง': 12,
 'ต้นไม้': 13,
 'ป้าย': 14,
 'other': 15,
 'สัตว์จรจัด': 16}

In [ ]:
df["type"] = df["type"].apply(lambda x: type_to_idx[x])

In [ ]:
len(type_columns)

17

In [ ]:
df.head()

,type,comment
0,4,ขยะเยอะ
1,10,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
2,8,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
3,5,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
4,5,1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...


# Preprocess: comment

In [ ]:
df['comment'].fillna('', inplace=True) 
df['comment'] = df['comment'].astype(str) 

In [ ]:
# df = df.sample(2_000, random_state=123)

In [ ]:
import re
import string
import pythainlp
from tqdm.auto import tqdm
tqdm.pandas()

def preprocessing_comment(comment):
    # Normalize text
    try:
      comment = pythainlp.util.normalize(comment)
    except:
      print(comment)

    # Tokenize text
    tokens = pythainlp.word_tokenize(comment)

    # Remove stop words
    stopwords = pythainlp.corpus.common.thai_stopwords()
    tokens = [token for token in tokens if token not in stopwords]

    # Join tokens back into a string
    processed_comment = ' '.join(tokens)
  
    return comment
    # return processed_comment

df['comment'] = df['comment'].progress_apply(preprocessing_comment)

  0%|          | 0/208999 [00:00<?, ?it/s]

In [ ]:
# df = df.sample(30_000, random_state=123)

# Train Val Test split

In [ ]:
from sklearn.model_selection import train_test_split

# df, df_tmp = train_test_split(df, test_size=0.5, random_state=123)

df_train, df_rem = train_test_split(df, test_size=0.3, random_state=123)

df_val, df_test = train_test_split(df_rem, test_size=0.5, random_state=123)

# Preprocess torch

In [ ]:
import torch
from torch.utils.data import Dataset

class MultiLabelTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.feature = self.data["comment"]
        self.labels = self.data["type"].to_numpy()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        comment = self.data['comment'].iloc[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        # Tokenize the comment
        tokenized_comment = self.tokenizer(comment, padding='max_length', truncation=True, max_length=256, return_tensors='pt')

        return {
            'input_ids': tokenized_comment['input_ids'].squeeze(),
            'attention_mask': tokenized_comment['attention_mask'].squeeze(),
            'labels': label
        }


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Step 2: Prepare the data
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", use_fast=False)
# Process your data and convert it to input features (input_ids, attention_mask, labels_binary)
train_dataset = MultiLabelTextDataset(df_train, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataset = MultiLabelTextDataset(df_val, tokenizer)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataset = MultiLabelTextDataset(df_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
num_words = tokenizer.vocab_size
num_words

25005

In [ ]:
# train_dataset[6]

In [ ]:
# next(iter(train_loader))

# Model

In [ ]:
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
class Classifier(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Classifier, self).__init__()
        self.fc2 = nn.Linear(n_inputs, n_outputs)
        # self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        # x = self.dropout(x)
        x = self.fc2(x)
        return x

wangchanberta_model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", num_labels=512).to(device)
class Model(nn.Module):
    def __init__(self, classifier):
        super().__init__()
        # self.embeding = wangchanberta_model
        self.embeding = nn.EmbeddingBag(tokenizer.vocab_size+10, 512,padding_idx=1)
        # for param in self.embeding.parameters():
        #     param.requires_grad = False
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 512)
        self.dropout = nn.Dropout(0.1)
        self.classifier = classifier


    def update_freeze_embed(self, is_freeze):
        if is_freeze:
          for param in self.embeding.parameters():
              param.requires_grad = False
        else:
          for param in self.embeding.parameters():
              param.requires_grad = True
        
    def forward(self,input_ids, attention_mask):
        # x = self.embeding(input_ids=input_ids, attention_mask=attention_mask).logits
        x = self.embeding(input_ids)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        out = self.classifier(x)
        return out

classifier = Classifier(512, len(type_columns))
model = Model(classifier).to(device)
# model.update_freeze_embed(False)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [ ]:
# Step 4: Setup the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Model(
  (embeding): EmbeddingBag(25015, 512, mode='mean', padding_idx=1)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Classifier(
    (fc2): Linear(in_features=512, out_features=17, bias=True)
  )
)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('always')

optimizer = torch.optim.Adam(model.parameters(), lr=7e-4)
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=4, 
    gamma=0.7
    ) 
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    num_train_samples = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_correct += (torch.argmax(outputs, dim=1) == labels).sum().item()

        train_loss += loss.item() * input_ids.size(0)
        num_train_samples += input_ids.size(0)

    train_loss /= num_train_samples
    train_accuracy = train_correct / num_train_samples

    model.eval()
    valid_loss = 0
    valid_correct = 0
    num_valid_samples = 0
    valid_predictions = []
    valid_labels = []

    with torch.no_grad():
        for batch in tqdm(valid_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = criterion(outputs, labels)
            valid_correct += (torch.argmax(outputs, dim=1) == labels).sum().item()

            valid_loss += loss.item() * input_ids.size(0)
            num_valid_samples += input_ids.size(0)

            valid_predictions.extend(torch.argmax(outputs, dim=1).cpu().tolist())
            valid_labels.extend(labels.cpu().tolist())

    valid_loss /= num_valid_samples
    valid_accuracy = valid_correct / num_valid_samples
    valid_precision = precision_score(valid_labels, valid_predictions, average='micro')
    valid_recall = recall_score(valid_labels, valid_predictions, average='micro')
    valid_f1 = f1_score(valid_labels, valid_predictions, average='micro')

    print(f"Epoch: {epoch + 1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")
    print(f"Valid Loss: {valid_loss:.4f}, Accuracy: {valid_accuracy:.4f}, Precision: {valid_precision:.4f}, Recall: {valid_recall:.4f}, F1: {valid_f1:.4f}")
    print(f"lr: {scheduler.optimizer.param_groups[0]['lr']:.8f}")
    scheduler.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 980/980 [00:22<00:00, 43.87it/s]


Epoch: 1/3:
Train Loss: 0.4119, Accuracy: 0.8573
Valid Loss: 0.4010, Accuracy: 0.8630, Precision: 0.8630, Recall: 0.8630, F1: 0.8630
lr: 0.00070000


100%|██████████| 980/980 [00:21<00:00, 45.75it/s]


Epoch: 2/3:
Train Loss: 0.3051, Accuracy: 0.8906
Valid Loss: 0.3882, Accuracy: 0.8700, Precision: 0.8700, Recall: 0.8700, F1: 0.8700
lr: 0.00070000


100%|██████████| 980/980 [00:20<00:00, 48.50it/s]


Epoch: 3/3:
Train Loss: 0.2357, Accuracy: 0.9143
Valid Loss: 0.3908, Accuracy: 0.8745, Precision: 0.8745, Recall: 0.8745, F1: 0.8745
lr: 0.00070000


In [ ]:
# valid_loss = 0
# valid_correct = 0
# num_valid_samples = 0
# valid_predictions = []
# valid_labels = []

# with torch.no_grad():
#     for batch in tqdm(valid_loader):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)

#         loss = criterion(outputs, labels)
#         valid_correct += (torch.argmax(outputs, dim=1) == labels).sum().item()

#         valid_loss += loss.item() * input_ids.size(0)
#         num_valid_samples += input_ids.size(0)

#         valid_predictions.extend(torch.argmax(outputs, dim=1).cpu().tolist())
#         valid_labels.extend(labels.cpu().tolist())

# valid_loss /= num_valid_samples
# valid_accuracy = valid_correct / num_valid_samples
# valid_precision = precision_score(valid_labels, valid_predictions, average='micro')
# valid_recall = recall_score(valid_labels, valid_predictions, average='micro')
# valid_f1 = f1_score(valid_labels, valid_predictions, average='micro')
# print(f"Valid Loss: {valid_loss:.4f}, Accuracy: {valid_accuracy:.4f}, Precision: {valid_precision:.4f}, Recall: {valid_recall:.4f}, F1: {valid_f1:.4f}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(data_loader, model, criterion, device):
    valid_loss = 0
    valid_correct = 0
    num_valid_samples = 0
    valid_predictions = []
    valid_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = criterion(outputs, labels)
            valid_correct += (torch.argmax(outputs, dim=1) == labels).sum().item()

            valid_loss += loss.item() * input_ids.size(0)
            num_valid_samples += input_ids.size(0)

            valid_predictions.extend(torch.argmax(outputs, dim=1).cpu().tolist())
            valid_labels.extend(labels.cpu().tolist())

    valid_loss /= num_valid_samples
    valid_accuracy = valid_correct / num_valid_samples
    valid_precision = precision_score(valid_labels, valid_predictions, average='micro')
    valid_recall = recall_score(valid_labels, valid_predictions, average='micro')
    valid_f1 = f1_score(valid_labels, valid_predictions, average='micro')

    return valid_loss, valid_accuracy, valid_precision, valid_recall, valid_f1

evaluate_model(test_loader, model, criterion, device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 980/980 [00:32<00:00, 30.27it/s]


(0.39324250243687364,
 0.8743859649122807,
 0.8694145375125213,
 0.8743859649122807,
 0.8743859649122808)